In [1]:
import pandas as pd
data = pd.read_csv('C:/Users/Sardor.Maksudov/Documents/data.csv')
data['id'] = data['id'].astype(int)

In [7]:
import pandas as pd
import json
from datetime import datetime, timedelta

def tot_claim_cnt_l180d(contracts_json, application_date_str):
    if pd.isna(contracts_json) or contracts_json == '':
        return -3

    try:
        contracts = json.loads(contracts_json)
    except json.JSONDecodeError:
        return -3

    try:
        application_date = datetime.strptime(application_date_str, '%Y-%m-%d %H:%M:%S.%f%z')
    except ValueError:
        return -3

    start_date = application_date - timedelta(days=180)
    valid_claims_count = 0

    if isinstance(contracts, list):
        for contract in contracts:
            if 'claim_id' in contract and 'claim_date' in contract and contract['claim_id'] and contract['claim_date']:
                try:
                    claim_date = datetime.strptime(contract['claim_date'], '%d.%m.%Y').replace(tzinfo=application_date.tzinfo)
                    if start_date <= claim_date <= application_date:
                        valid_claims_count += 1
                except ValueError:
                    continue
    elif isinstance(contracts, dict):
        if 'claim_id' in contracts and 'claim_date' in contracts and contracts['claim_id'] and contracts['claim_date']:
            try:
                claim_date = datetime.strptime(contracts['claim_date'], '%d.%m.%Y').replace(tzinfo=application_date.tzinfo)
                if start_date <= claim_date <= application_date:
                    valid_claims_count = 1
            except ValueError:
                pass

    return valid_claims_count if valid_claims_count > 0 else -3






# Apply both functions to the DataFrame
data['tot_claim_cnt_l180d'] = data.apply(lambda row: tot_claim_cnt_l180d(row['contracts'], row['application_date']), axis=1)
# data['valid_claims_count_v2'] = data.apply(lambda row: count_valid_claims_v2(row['contracts'], row['application_date']), axis=1)

data

,id,application_date,contracts,tot_claim_cnt_l180d
0,2925210,2024-02-12 19:22:46.652000+00:00,NaN,-3
1,2925211,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",57
2,2925212,2024-02-12 19:24:41.493000+00:00,NaN,-3
3,2925213,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",57
4,2925214,2024-02-12 19:24:56.857000+00:00,NaN,-3
...,...,...,...,...
995,2926205,2024-02-13 06:09:54.210000+00:00,"[{""contract_id"": 18410, ""bank"": ""004"", ""summa""...",-3
996,2926206,2024-02-13 06:09:54.306000+00:00,NaN,-3
997,2926207,2024-02-13 06:09:55.661000+00:00,NaN,-3
998,2926208,2024-02-13 06:09:57.024000+00:00,NaN,-3


In [8]:
import pandas as pd
import json

def calculate_disb_bank_loan_wo_tbc(contracts_json):
    if pd.isna(contracts_json) or contracts_json == '':
        return -3  # No claims at all

    try:
        contracts = json.loads(contracts_json)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}, contracts_json: {contracts_json}")
        return -3  # No claims at all if JSON decoding fails

    # Initialize total exposure
    total_exposure = 0
    valid_loan_found = False
    claims_exist = False

    # Define the banks to exclude
    exclude_banks = ['LIZ', 'LOM', 'MKO', 'SUG', None]

    # Helper function to process each contract
    def process_contract(contract):
        nonlocal total_exposure, valid_loan_found, claims_exist
        if isinstance(contract, dict):
            # Check if claims exist
            if 'claim_id' in contract and contract['claim_id'] and 'claim_date' in contract and contract['claim_date']:
                claims_exist = True

            if (contract.get('bank') not in exclude_banks and 
                contract.get('contract_date') not in (None, '')):
                loan_summa = str(contract.get('loan_summa', '0')).strip()
                try:
                    loan_summa_value = float(loan_summa) if loan_summa else 0.0
                    total_exposure += loan_summa_value
                    valid_loan_found = True
                except ValueError as e:
                    print(f"Value error for loan_summa: {e}, loan_summa: {loan_summa}")

    # Check if contracts is a list of dictionaries
    if isinstance(contracts, list):
        if not contracts:
            return -3  # No claims at all
        for contract in contracts:
            process_contract(contract)
    # Check if contracts is a single dictionary
    elif isinstance(contracts, dict):
        process_contract(contracts)
    else:
        return -3  # No claims at all

    if not claims_exist:
        return -3  # No claims at all

    if not valid_loan_found:
        return -1  # No valid loans

    return total_exposure if total_exposure > 0 else -1

# Apply the function to each row in the 'contracts' column
data['disb_bank_loan_wo_tbc'] = data['contracts'].apply(calculate_disb_bank_loan_wo_tbc)

# Display the DataFrame to verify the new column
data.head(100)


,id,application_date,contracts,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc
0,2925210,2024-02-12 19:22:46.652000+00:00,NaN,-3,-3.0
1,2925211,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",57,-1.0
2,2925212,2024-02-12 19:24:41.493000+00:00,NaN,-3,-3.0
3,2925213,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",57,-1.0
4,2925214,2024-02-12 19:24:56.857000+00:00,NaN,-3,-3.0
...,...,...,...,...,...
95,2925305,2024-02-13 04:17:57.561000+00:00,"[{""contract_id"": """", ""bank"": ""LOM"", ""summa"": ""...",16,300000000.0
96,2925306,2024-02-13 04:18:18.516000+00:00,NaN,-3,-3.0
97,2925307,2024-02-13 04:18:23.176000+00:00,NaN,-3,-3.0
98,2925308,2024-02-13 04:18:24.316000+00:00,NaN,-3,-3.0


In [9]:
import pandas as pd
import json
from datetime import datetime

def calculate_day_sinlastloan(contracts_json, application_date_str):
    if pd.isna(contracts_json) or contracts_json == '':
        return -3  # No claims at all

    try:
        contracts = json.loads(contracts_json)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}, contracts_json: {contracts_json}")
        return -3  # Return -3 if JSON decoding fails

    try:
        application_date = datetime.strptime(application_date_str, '%Y-%m-%d %H:%M:%S.%f%z')
    except ValueError as e:
        print(f"Date parsing error: {e}, application_date_str: {application_date_str}")
        return -3  # Return -3 if date parsing fails

    # Check if contracts is a list of dictionaries or a single dictionary
    if isinstance(contracts, dict):
        contracts = [contracts]

    # Check if there are any claims
    claims_exist = any(
        'claim_id' in contract and contract['claim_id'] and 'claim_date' in contract and contract['claim_date']
        for contract in contracts
    )

    if not claims_exist:
        return -3  # No claims at all

    # Filter valid loans with non-null and non-empty summa and non-null contract_date
    valid_loans = [
        contract for contract in contracts
        if contract.get('summa') not in (None, '', '0') and contract.get('contract_date') not in (None, '')
    ]

    if not valid_loans:
        return -1  # No valid loans

    # Find the last loan based on the contract_date
    last_loan = None
    for contract in valid_loans:
        for date_format in ['%Y-%m-%d', '%d.%m.%Y']:
            try:
                contract_date = datetime.strptime(contract['contract_date'], date_format)
                if last_loan is None or contract_date > last_loan:
                    last_loan = contract_date
                break
            except ValueError:
                continue

    if last_loan is None:
        return -1  # No valid contract_date found

    # Convert last_loan to offset-aware datetime
    last_loan = last_loan.replace(tzinfo=application_date.tzinfo)

    # Calculate days since last loan
    days_since_last_loan = (application_date - last_loan).days

    return days_since_last_loan if days_since_last_loan >= 0 else -3

# Apply the function to each row in the DataFrame
data['day_sinlastloan'] = data.apply(
    lambda row: calculate_day_sinlastloan(row['contracts'], row['application_date']), axis=1
)
data

Date parsing error: time data '2024-02-13 06:05:18+00:00' does not match format '%Y-%m-%d %H:%M:%S.%f%z', application_date_str: 2024-02-13 06:05:18+00:00
Date parsing error: time data '2024-02-13 06:05:18+00:00' does not match format '%Y-%m-%d %H:%M:%S.%f%z', application_date_str: 2024-02-13 06:05:18+00:00


,id,application_date,contracts,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan
0,2925210,2024-02-12 19:22:46.652000+00:00,NaN,-3,-3.0,-3
1,2925211,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",57,-1.0,427
2,2925212,2024-02-12 19:24:41.493000+00:00,NaN,-3,-3.0,-3
3,2925213,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",57,-1.0,427
4,2925214,2024-02-12 19:24:56.857000+00:00,NaN,-3,-3.0,-3
...,...,...,...,...,...,...
995,2926205,2024-02-13 06:09:54.210000+00:00,"[{""contract_id"": 18410, ""bank"": ""004"", ""summa""...",-3,217320883.0,729
996,2926206,2024-02-13 06:09:54.306000+00:00,NaN,-3,-3.0,-3
997,2926207,2024-02-13 06:09:55.661000+00:00,NaN,-3,-3.0,-3
998,2926208,2024-02-13 06:09:57.024000+00:00,NaN,-3,-3.0,-3


In [11]:
data.to_excel('claim_modified.xlsx', index=False)